## Instructions

Create a project with code that solves the following questions. Please generate a repository in GitHub with the code and make all necessary commits. The use of any AI tool is forbidden.

1. (4 points) Generate a pipeline for processing a text. The pipeline will have the following porcesses: Sentence splitter, word tokenization, conver acronims with dots to acronims whitout dots with regex (U.S.A. to USA), remove stopwords, lematizer. You must do each step in pure python and with a NLP library of your choice.

2. (4 points) Create an application with user interface that call a local LLM to solve any problem you choose. Modify the system prompt to better solve the problem.

3. (2 points) Integrate in the application of the previous question with a speech model for speech to text and text to speech. The model should be from Huging Face.

Please upload the code files and a link to the github repository. The assesment will be with a one on one defense. 

## Part 1

Genarate a pipeline for processing a text.

the text I will be using to complete this task is as follows:

In [2]:
text = "The president of the U.S.A., Donald Trump, is 1.9m high and 78 years old. Forbes Magazine has assessed his wealth, currently estimating it at $5.5 billion as of mid-February 2025."

### Imports

In [4]:
import nltk
import re
from num2words import num2words
from collections import defaultdict

Using NLTK to split sentences

In [5]:
sentences = nltk.sent_tokenize(text)

print(sentences)

['The president of the U.S.A., Donald Trump, is 1.9m high and 78 years old.', 'Forbes Magazine has assessed his wealth, currently estimating it at $5.5 billion as of mid-February 2025.']


Using NLTK for word tokenization

In [7]:
words = nltk.word_tokenize(text)

print(words)

['The', 'president', 'of', 'the', 'U.S.A.', ',', 'Donald', 'Trump', ',', 'is', '1.9m', 'high', 'and', '78', 'years', 'old', '.', 'Forbes', 'Magazine', 'has', 'assessed', 'his', 'wealth', ',', 'currently', 'estimating', 'it', 'at', '$', '5.5', 'billion', 'as', 'of', 'mid-February', '2025', '.']


Convert acronyms with regex

In [8]:
text = re.sub(r'U\.S\.A\.', 'USA', text)

print(text)

The president of the USA, Donald Trump, is 1.9m high and 78 years old. Forbes Magazine has assessed his wealth, currently estimating it at $5.5 billion as of mid-February 2025.
